### ДЗ №5. Линейная регрессия

In [1]:
import numpy as np

SEED = 17
np.random.seed(SEED)

In [4]:
import pandas as pd
from sklearn import datasets

In [40]:
class SimpleLinearRegression:
    def __init__(self, step = 0.01, tol = 1e-4, max_iter=2000, verbose=False, random_state=SEED):
        self.max_iter = max_iter # max iter count of gradient descent
        self.step = step # step of descent in the direction of antigradient
        self.tol = tol # we compare norm of gradient with that threshold
        self._w = None # w_1
        self._intercept = None # w_0
        self.random_state = random_state 
        self.verbose = verbose
        
    def predict(self, X):
        """
        estimate target variable "y" based on features X 
        """
        y_pred = self._w*X + self._intercept #my code
        assert y_pred.shape[0] == X.shape[0]
        return y_pred

    def score(self, X, y):
        """
        MSE
        X - features
        y - true values of target variable
        """
        return np.mean((y - self.predict(X))**2)
    
    def _gradient(self, X, y):
        """
        Compute gradient of MSE subject to w_1, w_0
        X - features
        y - true values of target variable
        """
        grad_w = -np.mean((y-self._w*X-self._intercept)*X)
        grad_intercept = -np.mean(y-self._w*X-self._intercept)
        # YOUR CODE HERE
        return grad_w, grad_intercept
        
    def fit(self, X, y):
        """
        Train model with gradient descent
        X - features
        y - true values of target variable
        """
        # for reproducable results
        np.random.seed(self.random_state)
        
        # initialize weights
        self._w, self._intercept = np.random.randn(2)
        # perform gradient descent
        for iter in range(self.max_iter):
            # compute gradient at current W
            grad_w, grad_intercept = self._gradient(X, y)
            
            # make step, update W
            
            self._w = self._w - self.step*grad_w # YOUR CODE HERE
            self._intercept = self._intercept - self.step*grad_intercept# YOUR CODE HERE
            
            # compute gradient norm            
            grad_norm = np.sqrt(grad_w **2 + grad_intercept**2)# YOUR CODE HERE
            
            # people like to watch how the error is reducing during iterations 
            if self.verbose:
                mse_score = self.score(X, y)
                print('iteration %d, MSE = %f, ||grad|| = %f' % (iter, mse_score, grad_norm))
                
            # compare gradient norm with threshold
            if grad_norm < self.tol:
                print('model converged')
                return self
        print('model did not converge')
        return self

In [5]:
boston_data = datasets.load_boston()
boston_data.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR'])

In [48]:
df = pd.DataFrame(boston_data['data'], columns=boston_data['feature_names'])
df['target'] = boston_data['target']
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [13]:
# реализуем функцию, которая считает MSE 
def mse_score(y_true, y_pred):
    """
    y_true - true values of target variable
    y_pred - predicted values of target variable 
    """
    result = np.mean((y_true - y_pred)**2)
    return result

In [11]:
from sklearn.model_selection import train_test_split

# разбили датасет в соотношении 60:40
df_train, df_test = train_test_split(df, test_size=0.4, random_state=SEED, shuffle=True)

In [ ]:
# обучите модель на df_train c verbose=True
# Обратите внимание на отладочный вывод, ваша ошибка MSE должна уменьшаться с каждой итерацией
# мы хотим научится предсказывать значение target по признаку CRIM
model = SimpleLinearRegression(verbose=True)
model.fit(X=df_train['CRIM'], y=df_train['target'])
mse_train_score = mse_score(df_train['target'], model.predict(df_train['CRIM']))
print('MSE on train:', mse_train_score)

#### MSE on train: 73.31274157452383

In [ ]:
# # посчитали качество обученной модели на df_test
mse_test_score = mse_score(df_test['target'], model.predict(df_test['CRIM']))
print('MSE on test:', mse_test_score)

#### MSE on test: 74.28787061058355